# 4. Data Mining
We will address finally running the Regression algorithms. In particular we will use:
a. Regression Trees
b. Lasso Regression
c. Linear Regression.


## a. Regression Trees
The idea when using Regression Trees is to create two models, one using the unscaled data (since this methods does not require scaling), and one using the scaled data. The intuition is that the performance should be the same on both models.